# Manipulating Expressions in Julia

Let's work through more examples of making expressions.

## What's an Expr?

Expressions have the head-tail structure that's like M-expressions.

In [12]:
a=2;
b=3;

In [5]:
e = Expr(:*, :a, :b)

:($(Expr(:*, :a, :b)))

In [6]:
e.head

:*

In [7]:
e.args

2-element Array{Any,1}:
 :a
 :b

In [13]:
eval(e)

6

We can generate that same expression with the `:` quote, but the resulting AST looks different:

In [14]:
e = :(a*b)

:(a * b)

In [15]:
e.head

:call

In [16]:
e.args

3-element Array{Any,1}:
 :*
 :a
 :b

In [17]:
eval(e)

6

This shows us how call syntax and infix syntax for operators get's parsed exactly the same way:

In [18]:
e = :(*(a,b))

:(a * b)

In [19]:
e.head

:call

In [20]:
e.args

3-element Array{Any,1}:
 :*
 :a
 :b

Looks like by manually creating the `Expr` object, we skipped an extra `:call` node.

## Putting Expressions together

If we want to programatically generate code, we're going to end up with lists and generators of codeblocks. How do we put them together? We'll have to create the data structures manually.

Suppose we have two quotes in an array:

In [21]:
quotes = [
    quote
        a=1
    end,
    quote
        b=3
        println("hello")
    end
]

2-element Array{Expr,1}:
 quote
    #= In[21]:3 =#
    a = 1
end                                        
 quote
    #= In[21]:6 =#
    b = 3
    #= In[21]:7 =#
    println("hello")
end

The principal-of-least surprise dictates this should return a new `Expr` by analogy to strings and sets:

In [3]:
quotes[1] * quotes[2]

MethodError: MethodError: no method matching *(::Expr, ::Expr)
Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:502

I'm surprised. It took me a few hours to realize there is no support for this seemingly basic operation, and we need to build an expression by manipulating the structures ourselves.

Quotes are blocks, and their contents are in the args:

In [22]:
quotes[1].head

:block

In [23]:
quotes[1].args

2-element Array{Any,1}:
 :(#= In[21]:3 =#)
 :(a = 1)         

Note how quotes include comments.

Both quotes are blocks with arrays of expressions. We want to take in two blocks, and return one block.

In [41]:
quotes[2].args

4-element Array{Any,1}:
 :(#= In[21]:6 =#)  
 :(b = 3)           
 :(#= In[21]:7 =#)  
 :(println("hello"))

In [50]:
s = [q.args for q in quotes]

2-element Array{Array{Any,1},1}:
 [:(#= In[21]:3 =#), :(a = 1)]                                        
 [:(#= In[21]:6 =#), :(b = 3), :(#= In[21]:7 =#), :(println("hello"))]

In [51]:
vcat(s...)

6-element Array{Any,1}:
 :(#= In[21]:3 =#)  
 :(a = 1)           
 :(#= In[21]:6 =#)  
 :(b = 3)           
 :(#= In[21]:7 =#)  
 :(println("hello"))

We can do this as a one-liner using the argument-unpacking syntax `...` (I think it's called "splatting: in the Julia lingo):

In [58]:
e = Expr(:block, vcat([q.args for q in quotes]...)...)

quote
    #= In[21]:3 =#
    a = 1
    #= In[21]:6 =#
    b = 3
    #= In[21]:7 =#
    println("hello")
end

One-liners are not necessarily the best, but I'm betting that the `...` is quite fast. Besides, certain parts of metaprogramming does not need to be *fast*, since it's only run once. (Of course, I have some kernels that take up to 10 minutes to generate, but that's the symbolic operations to derive shape functions. I'm hoping Julia can solve that problem.)

We can check to make sure this works:

In [61]:
eval(e)
println(a," ",b)

hello
1 3


What if non-quote expressions are mixed in?

In [62]:
quotes2 = [
    quote
        a=12
    end,
    quote
        b=33
        println("hello")
    end,
    :(println(a+b)),
    :(c=a*b)
]

4-element Array{Expr,1}:
 quote
    #= In[62]:3 =#
    a = 12
end                                        
 quote
    #= In[62]:6 =#
    b = 33
    #= In[62]:7 =#
    println("hello")
end
 :(println(a + b))                                                              
 :(c = a * b)                                                                   

In [64]:
e = Expr(:block, vcat([q.args for q in quotes2]...)...)

quote
    #= In[62]:3 =#
    a = 12
    #= In[62]:6 =#
    b = 33
    #= In[62]:7 =#
    println("hello")
    println
    a + b
    c
    a * b
end

In [65]:
eval(e)

hello


UndefVarError: UndefVarError: c not defined

Something went wrong because we just stripped the args. We need to add some type-checking on the blocks. The more robust function function unifies expressions and blocks into one block. Let's instead apply a sanitizing step first:

In [79]:
strip(q::Expr) = (q.head == :block ? q.args : q )

strip (generic function with 1 method)

In [90]:
e = Expr(:block, vcat(map(strip, quotes2)...)...)

quote
    #= In[62]:3 =#
    a = 12
    #= In[62]:6 =#
    b = 33
    #= In[62]:7 =#
    println("hello")
    println(a + b)
    c = a * b
end

## Putting it together in an unsurprising way.

In [95]:
f(args...) = args

f (generic function with 1 method)

In [97]:
f(quotes)[2]

2-element Array{Expr,1}:
 quote
    #= In[21]:3 =#
    a = 1
end                                        
 quote
    #= In[21]:6 =#
    b = 3
    #= In[21]:7 =#
    println("hello")
end

In [99]:
ExprCat(quotes...) = Expr(:block, vcat([(q.head == :block ? q.args : q ) for q in quotes]...)...)

ExprCat (generic function with 1 method)

Now we can splat into it:

In [100]:
ExprCat(quotes...)

quote
    #= In[21]:3 =#
    a = 1
    #= In[21]:6 =#
    b = 3
    #= In[21]:7 =#
    println("hello")
end

But this doesn't work: Should it?

In [102]:
ExprCat(quotes)

ErrorException: type Array has no field head

We can add an array override to help with that principal of least surprise. It makes sense to use arrays of expressions/quotes interchangably with explicit it quote structs internally.

In [128]:
sanitize_quotes(q::Expr) = (q.head == :block ? q.args : q )
ExprCat(quotes::Array) = ExprCat(quotes...)
ExprCat(quotes...) = Expr(:block, vcat(map(sanitize_quotes,quotes)...)...)

ExprCat (generic function with 3 methods)

In [118]:
ExprCat(quotes)

quote
    #= In[21]:3 =#
    a = 1
    #= In[21]:6 =#
    b = 3
    #= In[21]:7 =#
    println("hello")
end

In [119]:
ExprCat(quotes2)

quote
    #= In[62]:3 =#
    a = 12
    #= In[62]:6 =#
    b = 33
    #= In[62]:7 =#
    println("hello")
    println(a + b)
    c = a * b
end

## Maximizing unsurprise

Quotes **should** join together exactly like Strings. So,

In [111]:
import Base.*
*(quote1::Expr,quote2::Expr) = ExprCat(quote1,quote2)

* (generic function with 347 methods)

We can also include typematching into varargs:

In [126]:
*(quotes::Expr...) = ExprCat(quotes...)

* (generic function with 348 methods)

That's a lot of methods using this name! Maybe we don't want to do this, but it gives us the most stylistically-Julia way of joining expressions:

In [112]:
quotes[1] * quotes[2]

quote
    #= In[21]:3 =#
    a = 1
    #= In[21]:6 =#
    b = 3
    #= In[21]:7 =#
    println("hello")
end

In [114]:
foldr(*, quotes2)

quote
    #= In[62]:3 =#
    a = 12
    #= In[62]:6 =#
    b = 33
    #= In[62]:7 =#
    println("hello")
    println(a + b)
    c = a * b
end

In [115]:
quotes[1] * quotes[2] * quotes2[3]

quote
    #= In[21]:3 =#
    a = 1
    #= In[21]:6 =#
    b = 3
    #= In[21]:7 =#
    println("hello")
    println(a + b)
end

We can splat into the operator with our varargs typing, but I think any more aggressive overloading would be too reckless.

In [127]:
*(quotes2...)

quote
    #= In[62]:3 =#
    a = 12
    #= In[62]:6 =#
    b = 33
    #= In[62]:7 =#
    println("hello")
    println(a + b)
    c = a * b
end

## Summing it all up:

We maximized co-suprise and came up with the must Julia-esque way to join together expressions into blocks. It's all just 6 lines code, which are now in the accompanyng Julia module.

In [129]:
sanitize_quotes(q::Expr) = (q.head == :block ? q.args : q )
ExprCat(quotes::Array) = ExprCat(quotes...)
ExprCat(quotes...) = Expr(:block, vcat(map(sanitize_quotes,quotes)...)...)
import Base.*
*(quote1::Expr,quote2::Expr) = ExprCat(quote1,quote2)
*(quotes::Expr...) = ExprCat(quotes...)

* (generic function with 348 methods)